# Amazon SageMaker Model - Bias Monitor


## Section 1 - Setup <a id='setup'></a>

#### 1.1 Import necessary libraries

In [7]:
%%time

from datetime import datetime, timedelta, timezone
import json
import os
import re
import boto3
from time import sleep
from threading import Thread

import pandas as pd

from sagemaker import get_execution_role, session, Session, image_uris
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.processing import ProcessingJob
from sagemaker.serializers import CSVSerializer

from sagemaker.model import Model
from sagemaker.model_monitor import (
    BiasAnalysisConfig,
    CronExpressionGenerator,
    DataCaptureConfig,
    EndpointInput,
    ExplainabilityAnalysisConfig,
    ModelBiasMonitor,
    ModelExplainabilityMonitor,
)

from sagemaker.clarify import (
    BiasConfig,
    DataConfig,
    ModelConfig,
    ModelPredictedLabelConfig,
    SHAPConfig,
)

CPU times: user 50 μs, sys: 5 μs, total: 55 μs
Wall time: 58.9 μs


#### 1.2 AWS region and IAM Role & Sagemaker Clients and Bucket Details

In [8]:
sagemaker_session = Session()
sagemaker_client = sagemaker_session.sagemaker_client
sagemaker_runtime_client = sagemaker_session.sagemaker_runtime_client

# Retrieve the default Amazon S3 bucket associated with the SageMaker session.
bucket = sagemaker_session.default_bucket()
print("Bucket:", bucket)

# Get the IAM role associated with the current SageMaker notebook or environment.
role = get_execution_role()
print("RoleArn:", role)

# Get the AWS region name for the current session.
region = boto3.Session().region_name
print("Region", region)

# Retrieve the AWS account ID of the caller using the Security Token Service (STS) client.
account_id = boto3.client("sts").get_caller_identity().get("Account")

# Create a Boto3 client for the SageMaker service, specifying the AWS region.
sm = boto3.Session().client(service_name="sagemaker", region_name=region)

file_key = "aai-540-group-3-final-project/data/eda/data.csv"

# Create an S3 client
s3 = boto3.client('s3')

Bucket: sagemaker-us-east-1-796598873577
RoleArn: arn:aws:iam::796598873577:role/LabRole
Region us-east-1


#### 1.3 S3 bucket and prefixes

In [9]:
# Bucket prefix to store details for the monitor
prefix = "sagemaker/clarify-bias-monitor"

# Other prefixes
data_capture_prefix = f"{prefix}/datacapture"
s3_capture_upload_path = f"s3://{bucket}/{data_capture_prefix}"

ground_truth_upload_path = (
    f"s3://{bucket}/{prefix}/ground_truth_data/{datetime.now():%Y-%m-%d-%H-%M-%S}"
)

reports_prefix = f"{prefix}/reports"
s3_report_path = f"s3://{bucket}/{reports_prefix}"

##Get the model monitor image
monitor_image_uri = image_uris.retrieve(framework="model-monitor", region=region)

print("Image URI:", monitor_image_uri)
print(f"Capture path: {s3_capture_upload_path}")
print(f"Ground truth path: {ground_truth_upload_path}")
print(f"Report path: {s3_report_path}")

Image URI: 156813124566.dkr.ecr.us-east-1.amazonaws.com/sagemaker-model-monitor-analyzer
Capture path: s3://sagemaker-us-east-1-796598873577/sagemaker/clarify-bias-monitor/datacapture
Ground truth path: s3://sagemaker-us-east-1-796598873577/sagemaker/clarify-bias-monitor/ground_truth_data/2025-02-16-22-01-43
Report path: s3://sagemaker-us-east-1-796598873577/sagemaker/clarify-bias-monitor/reports


#### 1.4 Test access to the S3 bucket
Let's quickly verify that the notebook has the right permissions to access the S3 bucket specified above.
Upload a simple test object into the S3 bucket.  If this command fails, the data capture and model monitoring capabilities will not work from this notebook.  You can fix this by updating the role associated with this notebook instance to have "s3:PutObject" permissions and try this validation again

In [13]:
! ls -la 

total 100
drwxr-xr-x 3 nobody users    6144 Feb 16 22:03 .
drwxr-xr-x 9 nobody users    6144 Feb 16 20:53 ..
drwxr-xr-x 2 nobody nogroup  6144 Feb 16 22:03 .ipynb_checkpoints
-rw-r--r-- 1 nobody nogroup     0 Feb 16 22:03 README.md
-rw-r--r-- 1 nobody users       0 Jan 26 21:38 __init__.py
-rw-r--r-- 1 nobody nogroup 81346 Feb 16 22:02 bias_monitor.ipynb


In [14]:
# Upload some test files
S3Uploader.upload("README.md", f"s3://{bucket}/test_upload")
print("Success! You are all set to proceed.")

Success! You are all set to proceed.


In [17]:
# Setup variables 

# TODO MODIFY MODEL ENTRY 
model_file = "xgboost_model.joblib"
# test_dataset = "test_data/test-dataset-input-cols.csv"
# validation_dataset = "test_data/validation-dataset-with-header.csv"
dataset_type = "text/csv"

## Section 2 - Deploy pre-trained model with data capture enabled <a id='deploy'></a>

In this section, you will upload the pretrained model to the S3 bucket, create an Amazon SageMaker Model, create an Amazon SageMaker real time endpoint, and enable data capture on the endpoint to capture endpoint invocations, predictions, and metadata.

#### 2.1 Upload the pre-trained model to S3

This code uploads a pre-trained XGBoost model that is ready for you to deploy. This model was trained using the XGB Churn Prediction Notebook in SageMaker. You can also use your own pre-trained model in this step. If you already have a pretrained model in Amazon S3, you can add it instead by specifying the s3_key.


In [24]:
import tarfile
import os

# Save model as TAR.GZ
# Path to the saved model
model_file = 'xgboost_model.joblib'

# Create a directory to store the .tar.gz file if it doesn't already exist
output_dir = 'model_archive'
os.makedirs(output_dir, exist_ok=True)

# Define the path for the .tar.gz file
tar_gz_file = f'{output_dir}/xgboost_model.tar.gz'

# Create the .tar.gz file
with tarfile.open(tar_gz_file, 'w:gz') as tar:
    tar.add(model_file, arcname=os.path.basename(model_file))

print(f'Model archived as: {tar_gz_file}')

Model archived as: model_archive/xgboost_model.tar.gz


In [25]:
## Upload the pretrained model to S3

model_file_archive = "model_archive/xgboost_model.tar.gz"
s3_key = f"s3://{bucket}/{prefix}"
model_url = S3Uploader.upload(model_file_archive, s3_key)
model_url

's3://sagemaker-us-east-1-796598873577/sagemaker/clarify-bias-monitor/xgboost_model.tar.gz'

#### 2.2 Create SageMaker Model entity

This step creates an Amazon SageMaker model from the  model file uploaded to S3.

In [27]:
# Define the pre-built XGBoost container URI for the region you are working in
image_uri = image_uris.retrieve("xgboost", region=region, version="1.0-1")

# Create a Model object
model = Model(
    image_uri=image_uri,  # Use the prebuilt XGBoost container image
    model_data=model_url,
    role=role
)

#### 2.3 Deploy the model with data capture enabled.
Next, deploy the SageMaker model on a specific instance with data capture enabled.

In [ ]:
endpoint_name = f"xgb-attrition-model-bias-monitor-{datetime.utcnow():%Y-%m-%d-%H%M}"
print("EndpointName =", endpoint_name)

data_capture_config = DataCaptureConfig(
    enable_capture=True, sampling_percentage=100, destination_s3_uri=s3_capture_upload_path
)

model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name=endpoint_name,
    data_capture_config=data_capture_config,
)

EndpointName = xgb-attrition-model-bias-monitor-2025-02-16-2231
--

## 3 Setup Bias Monitor

In [41]:
model_bias_monitor = ModelBiasMonitor(
    role=role,
    sagemaker_session=sagemaker_session,
    max_runtime_in_seconds=1800,
)

model_bias_config = BiasConfig(
    label_values_or_threshold=[1],
    facet_name="Account Length", # sensitive feature to check for bias
    facet_values_or_threshold=[100],
)

model_bias_analysis_config = BiasAnalysisConfig(
    model_bias_config,
    headers=all_headers,
    label=label_header,
)

### Setup Monitor Schedule

In [46]:
# every hour
schedule_expression = CronExpressionGenerator.hourly()


model_bias_monitor.create_monitoring_schedule(
    analysis_config=model_bias_analysis_config,
    output_s3_uri=s3_report_path,
    endpoint_input=EndpointInput(
        endpoint_name=endpoint_name,
        destination="/opt/ml/processing/input/endpoint",
        start_time_offset="-PT1H",
        end_time_offset="-PT0H",
        probability_threshold_attribute=0.8,
    ),
    ground_truth_input=ground_truth_upload_path,
    schedule_cron_expression=schedule_expression,
)
print(f"Model bias monitoring schedule: {model_bias_monitor.monitoring_schedule_name}")

Model bias monitoring schedule: monitoring-schedule-2025-02-11-07-32-59-785


In [ ]:
# restart schedule if needed 

# model_bias_monitor.stop_monitoring_schedule()

# model_bias_monitor.start_monitoring_schedule()

In [47]:
# Check for start 
def wait_for_execution_to_start(model_monitor):
    print(
        "A hourly schedule was created above and it will kick off executions ON the hour (plus 0 - 20 min buffer)."
    )

    print("Waiting for the first execution to happen", end="")
    schedule_desc = model_monitor.describe_schedule()
    while "LastMonitoringExecutionSummary" not in schedule_desc:
        schedule_desc = model_monitor.describe_schedule()
        print(".", end="", flush=True)
        sleep(60)
    print()
    print("Done! Execution has been created")

    print("Now waiting for execution to start", end="")
    while schedule_desc["LastMonitoringExecutionSummary"]["MonitoringExecutionStatus"] in "Pending":
        schedule_desc = model_monitor.describe_schedule()
        print(".", end="", flush=True)
        sleep(10)

    print()
    print("Done! Execution has started")
        
wait_for_execution_to_start(model_bias_monitor)

A hourly schedule was created above and it will kick off executions ON the hour (plus 0 - 20 min buffer).
Waiting for the first execution to happen............................
Done! Execution has been created
Now waiting for execution to start.
Done! Execution has started


In [53]:
# Waits for the schedule to have last execution in a terminal status.
def wait_for_execution_to_finish(model_monitor):
    schedule_desc = model_monitor.describe_schedule()
    execution_summary = schedule_desc.get("LastMonitoringExecutionSummary")
    if execution_summary is not None:
        print("Waiting for execution to finish", end="")
        while execution_summary["MonitoringExecutionStatus"] not in [
            "Completed",
            "CompletedWithViolations",
            "Failed",
            "Stopped",
        ]:
            print(".", end="", flush=True)
            sleep(60)
            schedule_desc = model_monitor.describe_schedule()
            execution_summary = schedule_desc["LastMonitoringExecutionSummary"]
        print()
        print("Done! Execution has finished")
    else:
        print("Last execution not found")
        
        
wait_for_execution_to_finish(model_bias_monitor)

Waiting for execution to finish.........
Done! Execution has finished


## Check Results

In [54]:
schedule_desc = model_bias_monitor.describe_schedule()
execution_summary = schedule_desc.get("LastMonitoringExecutionSummary")
if execution_summary and execution_summary["MonitoringExecutionStatus"] in [
    "Completed",
    "CompletedWithViolations",
]:
    last_model_bias_monitor_execution = model_bias_monitor.list_executions()[-1]
    last_model_bias_monitor_execution_report_uri = (
        last_model_bias_monitor_execution.output.destination
    )
    print(f"Report URI: {last_model_bias_monitor_execution_report_uri}")
    last_model_bias_monitor_execution_report_files = sorted(
        S3Downloader.list(last_model_bias_monitor_execution_report_uri)
    )
    print("Found Report Files:")
    print("\n ".join(last_model_bias_monitor_execution_report_files))
else:
    last_model_bias_monitor_execution = None
    print(
        "====STOP==== \n No completed executions to inspect further. Please wait till an execution completes or investigate previously reported failures."
    )

Report URI: s3://sagemaker-us-east-1-796598873577/sagemaker/clarify-bias-monitor/reports/DEMO-xgb-churn-model-bias-monitor-2025-02-11-0706/monitoring-schedule-2025-02-11-07-32-59-785/2025/02/11/08
Found Report Files:
s3://sagemaker-us-east-1-796598873577/sagemaker/clarify-bias-monitor/reports/DEMO-xgb-churn-model-bias-monitor-2025-02-11-0706/monitoring-schedule-2025-02-11-07-32-59-785/2025/02/11/08/analysis.json
 s3://sagemaker-us-east-1-796598873577/sagemaker/clarify-bias-monitor/reports/DEMO-xgb-churn-model-bias-monitor-2025-02-11-0706/monitoring-schedule-2025-02-11-07-32-59-785/2025/02/11/08/report.html
 s3://sagemaker-us-east-1-796598873577/sagemaker/clarify-bias-monitor/reports/DEMO-xgb-churn-model-bias-monitor-2025-02-11-0706/monitoring-schedule-2025-02-11-07-32-59-785/2025/02/11/08/report.ipynb
 s3://sagemaker-us-east-1-796598873577/sagemaker/clarify-bias-monitor/reports/DEMO-xgb-churn-model-bias-monitor-2025-02-11-0706/monitoring-schedule-2025-02-11-07-32-59-785/2025/02/11/08/r

## Clean up <a id='cleanup'></a>  

You can keep your endpoint running to continue capturing data. If you do not plan to collect more data or use this endpoint further, you should delete the endpoint to avoid incurring additional charges. Note that deleting your endpoint does not delete the data that was captured during the model invocations. That data persists in Amazon S3 until you delete it yourself.

But before that, you need to delete the schedule first.

In [ ]:
invoke_endpoint_thread.terminate()
ground_truth_thread.terminate()

In [ ]:
from sagemaker.predictor import Predictor

predictor = Predictor(endpoint_name, sagemaker_session=sagemaker_session)
model_monitors = predictor.list_monitors()
for model_monitor in model_monitors:
    model_monitor.stop_monitoring_schedule()
    wait_for_execution_to_finish(model_monitor)
    model_monitor.delete_monitoring_schedule()

In [ ]:
predictor.delete_endpoint()
predictor.delete_model()

## References

https://sagemaker-examples.readthedocs.io/en/latest/sagemaker_model_monitor/fairness_and_explainability/SageMaker-Model-Monitor-Fairness-and-Explainability.html